# Tidal plots for HLTC National Map polygons

**Summary**: Iterates through ITEMv2 polygons, extracts Landsat observations and produces plots of matching data

**Issues**: 

**Notes**: Tidal model code based on original by Claire Phillips.

In [8]:
# Import all modules
import os
import glob
import datacube
import fiona
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

from datacube.utils import geometry
from datacube.api.query import query_group_by
from otps import TimePoint, predict_tide
from dateutil.relativedelta import relativedelta
from matplotlib.ticker import FormatStrFormatter

dc = datacube.Datacube(app='Tidal HLTC plots')


def date_range(start_date, end_date, increment, period):
    
    """Generate dates seperated by given time increment/period"""
    
    result = []
    nxt = start_date
    delta = relativedelta(**{period:increment})
    while nxt <= end_date:
        result.append(nxt)
        nxt += delta
    return result


# Setup   
filepath='/g/data/r78/intertidal/GA_native_tidal_model.shp'
products = ['ls5_pq_albers', 'ls7_pq_albers', 'ls8_pq_albers'] 
time_period = ('1986-01-01', '2018-01-01')   # Global time range
ls7_slc_period = ('1986-01-01', '2003-05-01') # Removes SLC failure

# List of files 
hltc_files = glob.glob("/g/data/fk4/datacube/002/HLTC/HLTC_2_0/geotiff/COMPOSITE*.tif")

## Produce plots

In [ ]:
for hltc_file in hltc_files:
    
    try:

        # Extract metadata from file
        basename = os.path.basename(hltc_file)[0:-4]
        print("Processing {}".format(basename))
        stage, polygon, lon, lat, time_from, time_to = basename.split("_")[1:-2]

        # Extract year, month, days
        year_from, month_from, day_from = time_from[:4], time_from[4:6], time_from[6:]
        year_to, month_to, day_to = time_to[:4], time_to[4:6], time_to[6:]

        # Convert to datetime; take 1 second off "to" date to that the previous day is used for printing
        dt_from = dt.datetime(int(year_from), int(month_from), int(day_from))
        dt_to = dt.datetime(int(year_to), int(month_to), int(day_to)) - dt.timedelta(seconds = 1)

        # Convert to string
        string_from = "{}-{}-{}".format(dt_from.year, dt_from.month, dt_from.day)
        string_to = "{}-{}-{}".format(dt_to.year, dt_to.month, dt_to.day)


        ################################
        print("   Model tide heights") #
        ################################

        # For each hour between start and end of timeperiod, create list of datetimes
        start = dt.datetime.strptime(time_period[0], "%Y-%m-%d")
        end = dt.datetime.strptime(time_period[1], "%Y-%m-%d")
        all_times = date_range(start, end, 1, 'hours')

        # For each timestep in list, predict tide and add to list
        times_model = [dt.strftime('%Y-%m-%d') for dt in all_times]
        tp_model = [TimePoint(float(lon), float(lat), dt) for dt in all_times]
        tides_model = [tide.tide_m for tide in predict_tide(tp_model)]

        # Covert to dataframe of observed dates and tidal heights
        df2_model = pd.DataFrame({'Model_height': tides_model}, index=pd.DatetimeIndex(times_model))


        ###################################
        print("   Observed tide heights") #
        ###################################

        all_times_obs = list()

        # Open ITEM polygons
        with fiona.open(filepath) as Input:
            crs = geometry.CRS(str(Input.crs_wkt))

            # For each polygon, extract ID
            for feature in Input:

                Id = feature['properties']['ID']            

                # If polygon is selected polygon
                if Id == int(polygon):

                    # Take first geometry and and extract area covered for input to dc call
                    first_geometry = feature['geometry']
                    geom = geometry.Geometry(first_geometry, crs=crs)

                    # For each product:                                
                    for source in products:   

                        # Use entire time range unless LS7                
                        time_range = ls7_slc_period if source == 'ls7_pq_albers' else time_period

                        # Determine matching datasets for geom area and group into solar day
                        ds = dc.find_datasets(product=source, time=time_range, geopolygon=geom)
                        group_by = query_group_by(group_by='solar_day')
                        sources = dc.group_datasets(ds, group_by)

                        # If data is found, add time to list then sort
                        if len(ds) > 0:
                            all_times_obs.extend(sources.time.data.astype('M8[s]').astype('O').tolist()) 

                    # Discontinue loop if polygon is found
                    break


        # Calculate tide data from X-Y-time location 
        all_times_obs = sorted(all_times_obs)  
        times_obs = [dt.strftime('%Y-%m-%d') for dt in all_times_obs]
        tp_obs = [TimePoint(float(lon), float(lat), dt) for dt in all_times_obs]
        tides_obs = [tide.tide_m for tide in predict_tide(tp_obs)]

        # Covert to dataframe of observed dates and tidal heights
        df1_obs = pd.DataFrame({'Tide_height': tides_obs}, index=pd.DatetimeIndex(times_obs)) 


        #############################
        print("   Plotting output") #
        #############################

        # Filter observed data by time
        df1_obs_subset = df1_obs.loc[string_from:string_to]

        # Compute percentage tide height
        min_height=df1_obs_subset.Tide_height.min()
        max_height=df1_obs_subset.Tide_height.max()
        dr = max_height - min_height

        # Create dict of percentile values
        per10_dict = {perc:min_height + dr * perc * 0.01 for perc in range(0, 110, 10)}

        # Filter observed data by tidal stage, using 0-20 for low composites and 80-100 for high
        tide_from, tide_to = {'LOW': (0, 20), 'HIGH': (80, 100)}[stage]
        df1_obs_subset = df1_obs_subset[df1_obs_subset.Tide_height.between(per10_dict[tide_from], 
                                                                           per10_dict[tide_to])]

        # Plot setup
        fig = plt.figure(figsize=(12,4))
        plt.margins(0) 
        fig.axes[0].spines['right'].set_visible(False)
        fig.axes[0].spines['top'].set_visible(False)
        fig.axes[0].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
        plt.ylabel('Tide height (m)')
#         plt.title("{0}\nTidal range between {1:g} and {2:g}m, period {3} / {4}".format(basename, 
#                                                                                        per10_dict[tide_from],
#                                                                                        per10_dict[tide_to],
#                                                                                        string_from,
#                                                                                        string_to))

        # Plot observations and modelled values
        plt.plot(df2_model.index, df2_model.Model_height, 
                 color='gainsboro', linewidth=0.5, zorder=1, label = 'Modelled')

        # Plot output
        plt.scatter(df1_obs.index, df1_obs.Tide_height, 
                    s=4, color='silver', marker='o', zorder=2)
        plt.scatter(df1_obs_subset.index, df1_obs_subset.Tide_height, 
                    s=10, color='black', marker='o', zorder=3, label = 'Observed')

        # Save and plot
        plt.savefig('figures/hltc_plots/{}.jpg'.format(basename), 
                    bbox_inches="tight", pad_inches=0.05, dpi = 150)
        plt.close()
    
    except:
        print("Skipping")
    

Processing COMPOSITE_HIGH_171_115.11_-21.16_20000101_20170101_PER_20
   Model tide heights
   Observed tide heights


In [2]:
hltc_files[50]

'/g/data/fk4/datacube/002/HLTC/HLTC_2_0/geotiff/COMPOSITE_HIGH_209_122.2_-18.07_20000101_20170101_PER_20.tif'